https://nvbn.github.io/2016/01/04/import-from-github/

https://www.python.org/dev/peps/pep-0302/

https://github.com/nvbn/import_from_github_com/blob/master/github_com/__init__.py

In [1]:
import sys
import pip
import os

class IntermediateModule:
    """Module for paths like `github_com.nvbn`."""

    def __init__(self, fullname):
        self.__package__ = fullname
        self.__path__ = fullname.split('.')
        self.__name__ = fullname


class Finder:
    """
    This method will be called with the fully qualified name of the module. If the finder is installed on sys.meta_path, it will receive a second argument, which is None for a top-level module, or package.__path__ for submodules or subpackages [5]. It should return a loader object if the module was found, or None if it wasn't. If find_module() raises an exception, it will be propagated to the caller, aborting the import.
    
    https://www.python.org/dev/peps/pep-0302/
    """

    def find_module(self, module_name, package_path):
        print("module_name is ", module_name)
        if module_name.startswith('piehub'):
            return Loader()


class Loader:
    """Installs and imports modules from github."""

    def _is_installed(self, fullname):
        try:
            self._import_module(fullname)
            return True
        except ImportError:
            return False

    def _import_module(self, fullname):
        """exists already so import.
        """
        actual_name = '.'.join(fullname.split('.')[2:])
        print("checking", actual_name)
        return __import__(actual_name)

    def _install_module(self, fullname):
        if not self._is_installed(fullname):
            url = fullname.replace('.', '/').replace('piehub', 'git+https://github.com', 1)
            command = "pip install " + url
            print("command is", command)
            os.system(command)

            
            
    def _is_repository_path(self, fullname):
        return fullname.count('.') == 2

    def _is_intermediate_path(self, fullname):
        return fullname.count('.') < 2

    def load_module(self, fullname):
        """
        once the interpreter receives a loader object, it will call the loader's
        load_module method.
        """

        # scenario 1 - 
        if self._is_repository_path(fullname):
            print('assigning', fullname, 'to path 1')
            self._install_module(fullname)

        # scenario 2a - 
        if self._is_intermediate_path(fullname):
            print('assigning', fullname, 'to path 2a')
            module = IntermediateModule(fullname)
        # scenario 2b
        else:
            print('assigning', fullname, 'to path 2b')
            module = self._import_module(fullname)

        # all modules are placed in sys.modules, which is different from globals()
        sys.modules[fullname] = module

In [2]:
sys.meta_path.append(GithubComFinder())

In [3]:
from piehub.kennethreitz import requests

module_name is  piehub
assigning piehub to path 2
module_name is  piehub.kennethreitz
assigning piehub.kennethreitz to path 2
module_name is  piehub.kennethreitz.requests
assigning piehub.kennethreitz.requests to path 1
checking requests
module_name is  requests
fullname piehub.kennethreitz.requests
url is git+https://github.com/kennethreitz/requests
command is pip install git+https://github.com/kennethreitz/requests
assigning piehub.kennethreitz.requests to path 3
path 3 fname piehub.kennethreitz.requests
checking requests
module_name is  urllib3.packages.six.moves
module_name is  urllib3.packages.six.moves.http_client
module_name is  brotli
module_name is  urllib3.packages.six.moves.urllib
module_name is  urllib3.packages.six.moves.urllib.parse
module_name is  brotli
module_name is  simplejson


In [6]:
assert requests.get('https://github.com').status_code == 200

In [7]:
__import__("requests")

<module 'requests' from '/Users/vincenttang/anaconda3/lib/python3.7/site-packages/requests/__init__.py'>

In [4]:
__import__("simplejson")

module_name is  simplejson


ModuleNotFoundError: No module named 'simplejson'